# Test Redshift Evolution of SNe
---
Aim of the game here is to look at how the fits change as redshift is pushed higher. Where do we see dropouts and fit failures.

When testing for the paper, we find that the Bazin model does quite well, but cannot capture the tail, whereas the Karpenka model struggles to be constrained e.g.

### Bazin I-Band
![Bazin_BessellI](../plots/paper/paper_fig_4_Bazin09.png)
### Karpenka I-Band
![Karpenka_BessellI](../plots/paper/paper_fig_4_Karpenka12.png)

### Bazin B-Band
![Bazin_BessellB](../plots/paper/paper_fig_5_Bazin09.png)
### Karpenka B-Band
![Karpenka_BessellB](../plots/paper/paper_fig_5_Karpenka12.png)

---

## Plan

going to try and play with the uncertainties to try and stabilise the fit. At the moment, even passing starting params from one fit (that works) to the next, causes a failure.

We can try a few things.

* The lightcurve is currently normalised in the simulate part of the code. As part of this, an uncertainty of 1 is applied to each specphot point. This means that the error is the same as the height of the peak. I will reduce this to see if it becomes more stable.

* I can pass on parameters from a successful fit. I can query the fitparams to try and exclude areas of parameter space that are clearly causing spurious fits

___
### Reducing Assumed Sigma

So I reduced the assumed sigma from 1 to 0.1. This did not help.

### Karpenka I-Band
![Karpenka_BessellI](../plots/paper/paper_fig_4_Karpenka12_01_sigma.png)
### Karpenka B-Band
![Karpenka_BessellB](../plots/paper/paper_fig_5_Karpenka12_01_sigma.png)

* Maybe I need to change the priors and/or the initial guesses

* Maybe I can reduce the sigma a bit more?

### Karpenka I-Band
![Karpenka_BessellI](../plots/paper/paper_fig_4_Karpenka12_001_sigma.png)
### Karpenka B-Band
![Karpenka_BessellB](../plots/paper/paper_fig_5_Karpenka12_001_sigma.png)

Nope.
___

In [ ]:
# %matplotlib inline
%matplotlib notebook

try:
    reload  # Python 2.7
except NameError:
    try:
        from importlib import reload  # Python 3.4+
    except ImportError:
        from imp import reload  # Python 3.0 - 3.3

import os

from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MultipleLocator

from collections import OrderedDict

rc('text', usetex=True)

import sfdmap

from astropy.table import Table,Column

In [ ]:
import json
import pandas as pd
import numpy as np

import pycoco as pcc
import pyCoCo as pccsim

from astropy.cosmology import LambdaCDM
from scipy.interpolate import InterpolatedUnivariateSpline

import lsst_tools as lsstt
from lcsim.simlib import SIMLIBReader
from lcsim.lcsim import LCSim
from datetime import datetime

In [ ]:
snname = "SN1998bw"

sn = pcc.classes.SNClass(snname)
print(sn.name)
sn.load_phot(verbose = True)

sn.load_list("/Users/berto/Code/CoCo/lists/"+snname+".list")
sn.load_spec()
sn.get_lcfit("/Users/berto/Code/CoCo/recon/"+snname+".dat")

sn.check_overlaps()
sn.load_mangledspec()
sn.get_simplespecphot()

sn.phot.data_filters["BessellB"]._plot_colour = pcc.colours.hex["BessellB"]

In [ ]:
sn.lcfit

In [ ]:
yaxis_lim_multiplier=1.1
verbose=False
lw=2.5
markersize=60.0
xminorticks=5.
yminorticks=0.5

pcc.utils.setup_plot_defaults()

params_dict = OrderedDict()

In [ ]:

fltPath = b"/Users/berto/Code/CoCo/data/filters"
rootPath = b"/Users/berto/Code/CoCo"

coco = pccsim.pyCoCo(fltPath, rootPath)

mjdmax = pcc.utils.get_mjdmax(sn, "BessellB")
mjdmax_to_sim = 0

path_to_filter = os.path.join(pcc.defaults._default_filter_dir_path,"BessellB.dat")  
BessellB = pcc.classes.FilterClass()
BessellB.read_filter_file(path_to_filter)
BessellB.calculate_AB_zp()

lcs = LCSim()

info = pcc.classes.InfoClass()
info.load()

mjd_list = np.arange(0.,95,0.1)-30.
mjd_to_sim = np.array([])
filters_to_sim = []

for mjd in mjd_list:
    filters_to_sim = np.append(filters_to_sim, lsstt.utils.b(BessellB.filter_name))
    mjd_to_sim = np.append(mjd_to_sim, mjd)

###
colourmap = plt.get_cmap("plasma")
fig = plt.figure(figsize=[12, 8])

fig.subplots_adjust(left = 0.09, bottom = 0.13, top = 0.99,
                    right = 0.97, hspace=0, wspace = 0)
ax1 = fig.add_subplot(111)
xaxis_label_string = r"$\textnormal{Phase, }\tau$"
yaxis_label_string = r"$\textnormal{Apparent B-band Magnitude, }m^{\textnormal{AB}}_{B}$"

z_sim = info.get_sn_info("SN1998bw")["z_distmod"]
z_list = [z_sim[0], 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]
# z_list = np.arange(0.0001,0.2,0.001)

log_z_list = np.log10(z_list)
z_cmap = log_z_list - np.nanmin(log_z_list)

z_cmap = z_cmap/np.nanmax(z_cmap)
for i, z_to_sim in enumerate(z_list):
    if verbose: print("running z =", z_to_sim)
    flux, flux_err = coco.simulate(b"SN1998bw",
                                   z_to_sim, 0.0, 0.0, 0.0, 3.1,
                                   mjdmax_to_sim, mjd_to_sim,
                                   filters_to_sim)
    specphot = coco.spec_photometry(b"SN1998bw", 
                 z_to_sim, b"BessellB")
    params = np.array(coco.get_fit_params()[b"BessellB"])
    if i == 0:
        params_array = params
    else:
        params_array = np.vstack((params_array, params))
    params_dict[str(z_to_sim)] = params
    ###

    sn_sim = pcc.classes.SNClass("SN1998bw")
    sn_sim.load_phot(phot_table = pcc.utils.simulate_out_to_ap_table(mjd_to_sim, flux, flux_err, filters_to_sim))

    sp = pcc.classes.PhotometryClass()
    sp.load_table(pcc.utils.specphot_out_to_ap_table(specphot, mjdmax_to_sim, "BessellB"))
    sp.phot["m_AB"] = -2.5 * np.log10(sp.phot["flux"]) - BessellB.zp_AB

    time_dilation = 1.+z_to_sim

    mag = -2.5 * np.log10(flux) - BessellB.zp_AB
#         ax1.scatter(sp.phot["MJD"], sp.phot["flux"])
    if i == 0:
        ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["m_AB"], color = pcc.colours.hex["BessellB"], 
                    label = r"$\textnormal{Bessell B}$", edgecolors = pcc.colours.hex["batman"], 
                    s=markersize)
        ax1.plot(mjd_to_sim*time_dilation, mag, color = colourmap(z_cmap[i]), label = r"$"+str(round(z_to_sim, 4))+"$",
                 lw=lw, zorder=0)
    else:
        ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["m_AB"], color = pcc.colours.hex["BessellB"],
                    label = None, edgecolors = pcc.colours.hex["batman"], s=markersize)
        ax1.plot(mjd_to_sim*time_dilation, mag, color = colourmap(z_cmap[i]), label = r"$"+str(z_to_sim)+"$",
                 lw=lw, zorder=0)

#         ax1.plot(mjd_to_sim, mag, color = "black", label = r"$"+str(z_to_sim)+"$")
    if verbose: print(r"$"+str(z_to_sim)+"$")
#         ax1.plot(mjd_to_sim, mag, color = colourmap(z_to_sim), label = str(z_to_sim))

#     ax1.set_ylim(np.nanmin(sp.phot["flux"]), np.nanmax(sp.phot["flux"])*yaxis_lim_multiplier)
#     ax1.set_ylim(0, 4.5e-15)
ax1.set_ylim(25.5, 13.)
ax1.set_xlim(-32.5, 82.5)

xminorLocator = MultipleLocator(xminorticks)
ax1.xaxis.set_minor_locator(xminorLocator)
yminorLocator = MultipleLocator(yminorticks)
ax1.yaxis.set_minor_locator(yminorLocator)

ax1.set_xlabel(xaxis_label_string)
ax1.set_ylabel(yaxis_label_string)
ax1.legend(loc = 1, scatterpoints = 1,
          numpoints = 1, frameon = False, fontsize = 14)
plt.show

In [ ]:
# params_array = params_array.reshape(len(params), len(z_list))

In [ ]:
params_array[:, 0]

In [ ]:
verbose = False

k = len(params_array.T)
dim=12
fig, axes = plt.subplots(k, k, figsize=(dim, dim))

for i, params in enumerate(params_array.T):
    if verbose: print("i", i,)
    for j in range(k):
        if verbose: print("j", j)
        ax = axes[j, i]
        if i > j:
            ax.set_frame_on(False)
            ax.set_xticks([])
            ax.set_yticks([])
        else:
#             ax.scatter(np.random.random(10), np.random.random(10))
            w = np.logical_and(np.isfinite(params_array[:, i]), np.isfinite(params_array[:, j]))
            ax.scatter(params_array[:, j][w], params_array[:, i][w])
            ax.set_xlim(np.nanmin(params_array[:, j][w]), np.nanmax(params_array[:, j][w]))
            ax.set_ylim(np.nanmin(params_array[:, i][w]), np.nanmax(params_array[:, i][w]))            

In [ ]:
plot_flux = False
# plot_flux = True

fltPath = b"/Users/berto/Code/CoCo/data/filters"
rootPath = b"/Users/berto/Code/CoCo"

coco = pccsim.pyCoCo(fltPath, rootPath)

mjdmax = pcc.utils.get_mjdmax(sn, "BessellB")
mjdmax_to_sim = 0

path_to_filter = os.path.join(pcc.defaults._default_filter_dir_path,"BessellB.dat")  
BessellB = pcc.classes.FilterClass()
BessellB.read_filter_file(path_to_filter)
BessellB.calculate_AB_zp()

lcs = LCSim()

info = pcc.classes.InfoClass()
info.load()

mjd_list = np.arange(0.,95,0.1)-30.
mjd_to_sim = np.array([])
filters_to_sim = []

for mjd in mjd_list:
    filters_to_sim = np.append(filters_to_sim, lsstt.utils.b(BessellB.filter_name))
    mjd_to_sim = np.append(mjd_to_sim, mjd)

###
colourmap = plt.get_cmap("plasma")
fig = plt.figure(figsize=[12, 8])

fig.subplots_adjust(left = 0.09, bottom = 0.13, top = 0.99,
                    right = 0.97, hspace=0, wspace = 0)
ax1 = fig.add_subplot(111)
xaxis_label_string = r"$\textnormal{Phase, }\tau$"
yaxis_label_string = r"$\textnormal{Apparent B-band Magnitude, }m^{\textnormal{AB}}_{B}$"

z_sim = info.get_sn_info("SN1998bw")["z_distmod"]
# z_list = [0.001, 0.002]
z_list = [0.002, 0.001]

# z_list = np.arange(0.0001,0.2,0.001)

log_z_list = np.log10(z_list)
z_cmap = log_z_list - np.nanmin(log_z_list)

z_cmap = z_cmap/np.nanmax(z_cmap)
for i, z_to_sim in enumerate(z_list):
    if verbose: print("running z =", z_to_sim)
    if i == 0: 
        flux, flux_err = coco.simulate(b"SN1998bw",
                                       z_to_sim, 0.0, 0.0, 0.0, 3.1,
                                       mjdmax_to_sim, mjd_to_sim,
                                       filters_to_sim)
    else:
        ## Try
#         params[0] = 1.02353411e-02
        flux, flux_err = coco.simulate_debug(b"SN1998bw",
                               z_to_sim, 0.0, 0.0, 0.0, 3.1,
                               mjdmax_to_sim, mjd_to_sim,
                               filters_to_sim, params[:-1]) ## NOTE - FINAL OUTPUT From SIMULATE coco.get_fit_params() is the NORMALISATION.
        
    specphot = coco.spec_photometry(b"SN1998bw", 
                 z_to_sim, b"BessellB")
    params = np.array(coco.get_fit_params()[b"BessellB"])
    if i == 0:
        params_array = params
    else:
        params_array = np.vstack((params_array, params))
    params_dict[str(z_to_sim)] = params
    ###

    sn_sim = pcc.classes.SNClass("SN1998bw")
    sn_sim.load_phot(phot_table = pcc.utils.simulate_out_to_ap_table(mjd_to_sim, flux, flux_err, filters_to_sim))

    sp = pcc.classes.PhotometryClass()
    sp.load_table(pcc.utils.specphot_out_to_ap_table(specphot, mjdmax_to_sim, "BessellB"))
    sp.phot["m_AB"] = -2.5 * np.log10(sp.phot["flux"]) - BessellB.zp_AB

    time_dilation = 1.+z_to_sim

    mag = -2.5 * np.log10(flux) - BessellB.zp_AB
#         ax1.scatter(sp.phot["MJD"], sp.phot["flux"])
    if i == 0:
        if plot_flux:
            ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["flux"], color = pcc.hex["BessellB"], 
                        label = r"$\textnormal{Bessell B}$", edgecolors = pcc.hex["batman"], 
                        s=markersize)
            ax1.plot(mjd_to_sim*time_dilation, flux, color = colourmap(z_cmap[i]), label = r"$"+str(round(z_to_sim, 4))+"$",
                     lw=lw, zorder=0)
        else:
            ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["m_AB"], color = pcc.hex["BessellB"], 
                        label = r"$\textnormal{Bessell B}$", edgecolors = pcc.hex["batman"], 
                        s=markersize)
            ax1.plot(mjd_to_sim*time_dilation, mag, color = colourmap(z_cmap[i]), label = r"$"+str(round(z_to_sim, 4))+"$",
                     lw=lw, zorder=0)

    else:
        if plot_flux:
            ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["flux"], color = pcc.hex["BessellB"],
                label = None, edgecolors = pcc.hex["batman"], s=markersize)
            ax1.plot(mjd_to_sim*time_dilation, flux, color = colourmap(z_cmap[i]), label = r"$"+str(z_to_sim)+"$",
                     lw=lw, zorder=0)
        else:
            ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["m_AB"], color = pcc.hex["BessellB"],
                        label = None, edgecolors = pcc.hex["batman"], s=markersize)
            ax1.plot(mjd_to_sim*time_dilation, mag, color = colourmap(z_cmap[i]), label = r"$"+str(z_to_sim)+"$",
                     lw=lw, zorder=0)
        


#         ax1.plot(mjd_to_sim, mag, color = "black", label = r"$"+str(z_to_sim)+"$")
    if verbose: print(r"$"+str(z_to_sim)+"$")
#         ax1.plot(mjd_to_sim, mag, color = colourmap(z_to_sim), label = str(z_to_sim))

#     ax1.set_ylim(np.nanmin(sp.phot["flux"]), np.nanmax(sp.phot["flux"])*yaxis_lim_multiplier)
if plot_flux:
    ax1.set_ylim(0, 1e-12)
else:
    ax1.set_ylim(17., 9.)
ax1.set_xlim(-32.5, 82.5)

xminorLocator = MultipleLocator(xminorticks)
ax1.xaxis.set_minor_locator(xminorLocator)
yminorLocator = MultipleLocator(yminorticks)
ax1.yaxis.set_minor_locator(yminorLocator)

ax1.set_xlabel(xaxis_label_string)
ax1.set_ylabel(yaxis_label_string)
ax1.legend(loc = 1, scatterpoints = 1,
          numpoints = 1, frameon = False, fontsize = 14)
plt.show

In [ ]:
axes[0, 0]

In [ ]:
def plot_params_hist(params):
    fig = plt.figure(figsize=[8, 6])

    fig.subplots_adjust(left = 0.09, bottom = 0.13, top = 0.99,
                        right = 0.97, hspace=0, wspace = 0)
    ax1 = fig.add_subplot(111)
    
    hist = ax1.hist(params, bins = 20)
    
def plot_2params(params1, params2):
    fig = plt.figure(figsize=[8, 6])

    fig.subplots_adjust(left = 0.09, bottom = 0.13, top = 0.99,
                        right = 0.97, hspace=0, wspace = 0)
    ax1 = fig.add_subplot(111)
    
    ax1.scatter(params1, params2)

In [ ]:
plot_params_hist(params_array[:, 0][np.isfinite(params_array[:, 0])])

In [ ]:
plot_params_hist(params_array[:,1][np.isfinite(params_array[:, 1])])

In [ ]:
plot_params_hist(params_array[:,2][np.isfinite(params_array[:,2])])

In [ ]:
plot_params_hist(params_array[:,3][np.isfinite(params_array[:,3])])

In [ ]:
plot_params_hist(params_array[:,4][np.isfinite(params_array[:,4])])

In [ ]:
plot_params_hist(params_array[:,5][np.isfinite(params_array[:,5])])

In [ ]:
plot_2params(params_array[:, 0][np.isfinite(params_array[:, 0])], params_array[:, 1][np.isfinite(params_array[:, 1])])

In [ ]:
plot_2params(params_array[:, 0][np.isfinite(params_array[:, 0])], params_array[:, 2][np.isfinite(params_array[:, 2])])

In [ ]:
plot_2params(params_array[:, 5][np.isfinite(params_array[:, 5])], params_array[:, 3][np.isfinite(params_array[:, 3])])

In [ ]:
# plot_flux = False
plot_flux = True

fltPath = b"/Users/berto/Code/CoCo/data/filters"
rootPath = b"/Users/berto/Code/CoCo"

coco = pccsim.pyCoCo(fltPath, rootPath)

mjdmax = pcc.utils.get_mjdmax(sn, "BessellB")
mjdmax_to_sim = 0

path_to_filter = os.path.join(pcc.defaults._default_filter_dir_path,"BessellB.dat")  
BessellB = pcc.classes.FilterClass()
BessellB.read_filter_file(path_to_filter)
BessellB.calculate_AB_zp()

lcs = LCSim()

info = pcc.classes.InfoClass()
info.load()

mjd_list = np.arange(0.,95,0.1)-30.
mjd_to_sim = np.array([])
filters_to_sim = []

for mjd in mjd_list:
    filters_to_sim = np.append(filters_to_sim, lsstt.utils.b(BessellB.filter_name))
    mjd_to_sim = np.append(mjd_to_sim, mjd)

###
colourmap = plt.get_cmap("plasma")
fig = plt.figure(figsize=[12, 8])

fig.subplots_adjust(left = 0.09, bottom = 0.13, top = 0.99,
                    right = 0.97, hspace=0, wspace = 0)
ax1 = fig.add_subplot(111)
xaxis_label_string = r"$\textnormal{Phase, }\tau$"
yaxis_label_string = r"$\textnormal{Apparent B-band Magnitude, }m^{\textnormal{AB}}_{B}$"

z_sim = info.get_sn_info("SN1998bw")["z_distmod"]
# z_list = [0.001, 0.002]
z_list = [0.002, 0.001]

# z_list = np.arange(0.0001,0.2,0.001)

log_z_list = np.log10(z_list)
z_cmap = log_z_list - np.nanmin(log_z_list)

z_cmap = z_cmap/np.nanmax(z_cmap)
for i, z_to_sim in enumerate(z_list):
    if verbose: print("running z =", z_to_sim)
    flux, flux_err = coco.simulate(b"SN1998bw",
                                   z_to_sim, 0.0, 0.0, 0.0, 3.1,
                                   mjdmax_to_sim, mjd_to_sim,
                                   filters_to_sim)
    specphot = coco.spec_photometry(b"SN1998bw", 
                 z_to_sim, b"BessellB")
    params = np.array(coco.get_fit_params())
    if i == 0:
        params_array = params
    else:
        params_array = np.vstack((params_array, params))
    params_dict[str(z_to_sim)] = params
    ###

    sn_sim = pcc.classes.SNClass("SN1998bw")
    sn_sim.load_phot(phot_table = pcc.utils.simulate_out_to_ap_table(mjd_to_sim, flux, flux_err, filters_to_sim))

    sp = pcc.classes.PhotometryClass()
    sp.load_table(pcc.utils.specphot_out_to_ap_table(specphot, mjdmax_to_sim, "BessellB"))
    sp.phot["m_AB"] = -2.5 * np.log10(sp.phot["flux"]) - BessellB.zp_AB

    time_dilation = 1.+z_to_sim

    mag = -2.5 * np.log10(flux) - BessellB.zp_AB
#         ax1.scatter(sp.phot["MJD"], sp.phot["flux"])
    if i == 0:
        if plot_flux:
            ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["flux"], color = pcc.hex["BessellB"], 
                        label = r"$\textnormal{Bessell B}$", edgecolors = pcc.hex["batman"], 
                        s=markersize)
            ax1.plot(mjd_to_sim*time_dilation, flux, color = colourmap(z_cmap[i]), label = r"$"+str(round(z_to_sim, 4))+"$",
                     lw=lw, zorder=0)
        else:
            ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["m_AB"], color = pcc.hex["BessellB"], 
                        label = r"$\textnormal{Bessell B}$", edgecolors = pcc.hex["batman"], 
                        s=markersize)
            ax1.plot(mjd_to_sim*time_dilation, mag, color = colourmap(z_cmap[i]), label = r"$"+str(round(z_to_sim, 4))+"$",
                     lw=lw, zorder=0)

    else:
        if plot_flux:
            ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["flux"], color = pcc.hex["BessellB"],
                label = None, edgecolors = pcc.hex["batman"], s=markersize)
            ax1.plot(mjd_to_sim*time_dilation, flux, color = colourmap(z_cmap[i]), label = r"$"+str(z_to_sim)+"$",
                     lw=lw, zorder=0)
        else:
            ax1.scatter(sp.phot["MJD"]*time_dilation, sp.phot["m_AB"], color = pcc.hex["BessellB"],
                        label = None, edgecolors = pcc.hex["batman"], s=markersize)
            ax1.plot(mjd_to_sim*time_dilation, mag, color = colourmap(z_cmap[i]), label = r"$"+str(z_to_sim)+"$",
                     lw=lw, zorder=0)
        


#         ax1.plot(mjd_to_sim, mag, color = "black", label = r"$"+str(z_to_sim)+"$")
    if verbose: print(r"$"+str(z_to_sim)+"$")
#         ax1.plot(mjd_to_sim, mag, color = colourmap(z_to_sim), label = str(z_to_sim))

#     ax1.set_ylim(np.nanmin(sp.phot["flux"]), np.nanmax(sp.phot["flux"])*yaxis_lim_multiplier)
if plot_flux:
    ax1.set_ylim(0, 1e-12)
else:
    ax1.set_ylim(17., 9.)
ax1.set_xlim(-32.5, 82.5)

xminorLocator = MultipleLocator(xminorticks)
ax1.xaxis.set_minor_locator(xminorLocator)
yminorLocator = MultipleLocator(yminorticks)
ax1.yaxis.set_minor_locator(yminorLocator)

ax1.set_xlabel(xaxis_label_string)
ax1.set_ylabel(yaxis_label_string)
ax1.legend(loc = 1, scatterpoints = 1,
          numpoints = 1, frameon = False, fontsize = 14)
plt.show

In [ ]:
for i, p in enumerate(params_array.T):
    print(p)

In [ ]:
"A"
"B"
"t1"
"t_r"
"t_f"
"t_0"

In [ ]:
for key in params_dict:
    print(params_dict[key])

In [ ]:
k = [key for key in params_dict]

In [ ]:
np.append([params_dict[k[0]]], [params_dict[k[1]]], axis=0).T

In [ ]:
print(mjdmax)
print(snname)

In [ ]:
# snname = "SN2007uy"
snname = snname.strip("b'")
# snname = "SN1993J"


sn = pcc.classes.SNClass(snname)

phot_path = os.path.join(pcc.defaults._default_data_dir_path, "lc/", snname + ".dat")
speclist_path = os.path.join(pcc.defaults._default_list_dir_path, snname + ".list")
recon_filename = os.path.join(pcc.defaults._default_recon_dir_path, snname + ".dat")

print(phot_path)
sn.load_phot(path = phot_path)
# sn.phot.plot()
sn.get_lcfit(recon_filename)

sn.load_list(path = speclist_path)
sn.load_spec()
# sn.load_mangledspec()
# sn.plot_spec()
# sn.plot_mangledspec()
# sn.plot_lc(multiplot = False, mark_spectra=True, savepng=True, outpath = "/Users/berto/projects/LSST/SN2007uy")
sn.plot_lc(multiplot = True, mark_spectra=True)
# sn.plot_lc(multiplot = False, mark_spectra=True)

In [ ]:
z_sim = 0.001
specphot_ap_table = Table()
# for filter_name in filter_names:
#     specphot = coco.spec_photometry(snname, 
#                      z_sim, lsstt.b(filter_name))
#     specphot_ap_table 
# specphot = coco.spec_photometry(lsstt.b(snname), 
#              z_sim, lsstt.b("SDSS_g"))
# specphot = coco.spec_photometry(lsstt.b(snname), 
#              z_sim, lsstt.b("SDSS_r"))
specphot = coco.spec_photometry(lsstt.b(snname), 
             z_sim, lsstt.b("BessellV"))
# specphot = coco.spec_photometry(lsstt.b(snname), 
#              z_sim, lsstt.b("BessellR"))

In [ ]:
specphot

In [ ]:
plt.scatter(specphot[0], specphot[1])
plt.ylim(np.nanmin(specphot[1]), np.nanmax(specphot[1])*1.1)

In [ ]:
verbose = False
stepsize = 0.05
plot = True
snname=pcc.utils.b(snname)
for z_sim in np.arange(0.001, z_max+stepsize,stepsize):
    print("simulating " , snname, "at z=", z_sim)
    if log:
        logpath = logfile + str(n_sne+1).rjust(6, "0") +".json"

    # snname = b"SN2011dh"
    # mag_offset = -2.0 ## Make Ia-like
    ## Simulate "Perfect" LC
    flux, flux_err = coco.simulate(snname,
                                   z_sim, mag_offset, MW_EBV, host_EBV, 3.1,
                                   mjdmax, mjd_to_sim,
                                   filters_to_sim)
    #
    # flux, flux_err = coco.simulate(snname,
    #                                z_obs, 0.0, 0.0, 0.0, 3.1,
    #                                mjdmax, mjd_to_sim,
    #                                filters_to_sim)
#         print(flux)
    p = pcc.classes.PhotometryClass()
    p.load_table(pcc.utils.simulate_out_to_ap_table(mjd_to_sim, flux, flux_err, filters_to_sim), verbose=False)
    if plot: p.plot(enforce_zero=True)

    ## calculate zeropoints and convert to mag
    p_df = p.phot.to_pandas()
    p_df["zp"] = p_df["filter"].map(zp_dict)
    if verbose: print(p_df)
    p_df["mag"] = -2.5 * np.log10(p_df.flux) - p_df.zp

    ## Add noise - returns units of
#         flux, flux_err = lcs.simulate(p_df["mag"], obslog, unit="ab")
    flux = pd.Series(flux)
    flux_err = pd.Series(0.05*flux)

#     if plot:
#         plt.errorbar(p_df.MJD, flux, yerr=flux_err.values, fmt="o")
#         plt.show()

    w_detected = np.where((~np.isnan(flux.values)) & ((flux.values/flux_err.values) > 5))[0]
    # if verbose:
    if verbose:
        print("Flux, Flux_err")
        print(flux, flux_err)
    if verbose:
        print("where detected, len")
        print(w_detected, len(w_detected))


    if verbose: print("good sne")
    ## Remove redshift simulated at top of code from the list
    # z_obs = np.delete(z_obs, [w_z])

    p_df["flux"] = flux
    p_df["flux_err"] = flux_err
    p_df["#MJD"] = p_df["MJD"]
    p_df.fillna(0, inplace = True)
    full_out_path = outfile + str(z_sim).rjust(6, "0") + ".dat"
    p_df[["#MJD", "flux", "flux_err", "filter"]].to_csv(full_out_path, sep=" ", index = False, )

    if log:
        logdict = {}
        for i in logvars:
            if type(locals()[i]) == np.ndarray:
                logdict[i] = locals()[i].tolist()
            elif type(locals()[i]) == np.int64:
                logdict[i] = int(locals()[i])
            elif type(locals()[i]) == pd.Series:
                logdict[i] = locals()[i].to_json()
            elif type(locals()[i]) == bytes:
                logdict[i] = str(locals()[i], "utf-8")
            else:
                logdict[i] = locals()[i]

        with open(logpath, "w") as ofile:
            json.dumps(logdict, sort_keys=True,
                       indent=4, separators=(',', ': '))
            #     for i in logvars:
            json.dump(logdict, ofile, sort_keys=True,
                      indent=4, separators=(',', ': '))

        #         ofile.write(str(i) + " " + str(locals()[i]) + "\n")
            ofile.close()


In [ ]:
-2.5 * np.log10(p_df.flux) - p_df.zp

Load in an templates object 